add volume thresholds and only trade when the underlying isnt going exponential against the spread positions Also only run a naive backtest

In [9]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from ib_insync import *
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

# --- PARAMETERS ---
numbars = 14400
durationStr = '7 D'
barSizeSetting = '5 mins'
symbol1 = 'TSM'
lastTradeDateOrContractMonth1 = '20250718'
strike1 = 230
right1 = 'C'    
symbol2 = 'TSM'
lastTradeDateOrContractMonth2 = '20250718'
strike2 = 240
right2 = 'C'

# --- FETCH FUNCTION ---
async def fetch_option(symbol, lastTradeDateOrContractMonth, strike, right, num_bars):
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7496, clientId=np.random.randint(1000, 10000))
    option_contract = Option(
        symbol=symbol,
        lastTradeDateOrContractMonth=lastTradeDateOrContractMonth,
        strike=strike,
        right=right,
        exchange='SMART',
        currency='USD'
    )
    await ib.qualifyContractsAsync(option_contract)
    all_bars = []
    end_time = ''
    while len(all_bars) < num_bars:
        bars = await ib.reqHistoricalDataAsync(
            contract=option_contract,
            endDateTime=end_time,
            durationStr=durationStr,
            barSizeSetting=barSizeSetting,
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1
        )
        if not bars:
            print("No more bars available.")
            break
        all_bars = bars + all_bars
        end_time = (bars[0].date - pd.Timedelta(minutes=1)).strftime('%Y%m%d %H:%M:%S')
        await asyncio.sleep(1)
    all_bars = all_bars[-num_bars:]
    df = util.df(all_bars)
    df.set_index('date', inplace=True)
    df.rename(columns={'close': 'Close'}, inplace=True)
    ib.disconnect()
    return df

# --- MAIN LOGIC ---
async def main():
    # Fetch option close prices
    datafix = await fetch_option(symbol1, lastTradeDateOrContractMonth1, strike1, right1, numbars)
    datafix2 = await fetch_option(symbol2, lastTradeDateOrContractMonth2, strike2, right2, numbars)
    datafix.index = datafix.index.tz_localize(None)
    datafix2.index = datafix2.index.tz_localize(None)

    # Calculate spread
    datafix["spread"] = datafix["Close"] - datafix2["Close"]

    # Calculate Bollinger Bands on Spread
    window = 20
    num_std = 2
    datafix['bb_middle'] = datafix['spread'].rolling(window=window).mean()
    datafix['bb_std'] = datafix['spread'].rolling(window=window).std()
    datafix['bb_upper'] = datafix['bb_middle'] + num_std * datafix['bb_std']
    datafix['bb_lower'] = datafix['bb_middle'] - num_std * datafix['bb_std']

    # Generate Buy/Sell/Hold Signals based on Bollinger Bands
    datafix['prev_spread'] = datafix['spread'].shift(1)
    datafix['prev_bb_lower'] = datafix['bb_lower'].shift(1)
    datafix['prev_bb_upper'] = datafix['bb_upper'].shift(1)
    buy_mask = (datafix['prev_spread'] > datafix['prev_bb_lower']) & (datafix['spread'] < datafix['bb_lower'])
    sell_mask = (datafix['prev_spread'] < datafix['prev_bb_upper']) & (datafix['spread'] > datafix['bb_upper'])
    datafix['bb_signal'] = 'hold'
    datafix.loc[buy_mask, 'bb_signal'] = 'buy'
    datafix.loc[sell_mask, 'bb_signal'] = 'sell'

    # --- Backtest with Delayed Entry/Exit (next close) and Single Position ---
    initial_capital = 1000
    capital = initial_capital
    position = 0  # 1 for long, -1 for short, 0 for flat
    positions = []
    portfolio = []
    entry_spread = None
    fee = 3  # dollar fee per trade

    pending_entry = None  # 'long', 'short', or None
    pending_exit = False  # True if we need to close on next bar

    for i, (dt, row) in enumerate(datafix.iterrows()):
        sig = row['bb_signal']
        spread = row['spread']

        # Execute pending exit if any
        if pending_exit:
            if position == 1:  # close long
                pnl = (spread - entry_spread) * 100
                capital += pnl
                capital -= fee
                position = 0
                entry_spread = None
            elif position == -1:  # close short
                pnl = (entry_spread - spread) * 100
                capital += pnl
                capital -= fee
                position = 0
                entry_spread = None
            pending_exit = False

        # Execute pending entry if any
        if pending_entry is not None and position == 0:
            if pending_entry == 'long':
                position = 1
                entry_spread = spread
                capital -= fee
            elif pending_entry == 'short':
                position = -1
                entry_spread = spread
                capital -= fee
            pending_entry = None

        # Set pending exit/entry based on current signal
        if position == 0:
            if sig == 'buy':
                pending_entry = 'long'
            elif sig == 'sell':
                pending_entry = 'short'
        elif position == 1 and sig == 'sell':
            pending_exit = True
        elif position == -1 and sig == 'buy':
            pending_exit = True

        portfolio.append(capital)
        positions.append(position)

    datafix['BB_Position'] = positions
    datafix['BB_Portfolio'] = portfolio

    total_return = datafix['BB_Portfolio'].iloc[-1] - initial_capital
    print(f"Total Strategy $ Return (Bollinger Bands): ${total_return:.2f}")

    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    # --- inside your main() function, replace the plotting block with this ---

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Spread and Bollinger Bands on primary y-axis (left)
    fig.add_trace(go.Scatter(
        x=datafix.index, y=datafix["spread"],
        mode='lines', name='Spread', line=dict(color='black', width=2)
    ), secondary_y=False)

    fig.add_trace(go.Scatter(
        x=datafix.index, y=datafix['bb_upper'],
        mode='lines', name='BB Upper', line=dict(color='green', dash='dash')
    ), secondary_y=False)

    fig.add_trace(go.Scatter(
        x=datafix.index, y=datafix['bb_lower'],
        mode='lines', name='BB Lower', line=dict(color='red', dash='dash')
    ), secondary_y=False)

    fig.add_trace(go.Scatter(
        x=datafix.index, y=datafix['bb_middle'],
        mode='lines', name='BB Middle', line=dict(color='orange', dash='dot')
    ), secondary_y=False)

    # Buy/Sell markers on primary y-axis
    buy_signals = datafix[datafix["bb_signal"] == "buy"]
    fig.add_trace(go.Scatter(
        x=buy_signals.index, y=buy_signals["spread"],
        mode='markers', name='Buy (on spread)',
        marker=dict(symbol='triangle-up', color='green', size=14, line=dict(color='black', width=1.5))
    ), secondary_y=False)

    sell_signals = datafix[datafix["bb_signal"] == "sell"]
    fig.add_trace(go.Scatter(
        x=sell_signals.index, y=sell_signals["spread"],
        mode='markers', name='Sell (on spread)',
        marker=dict(symbol='triangle-down', color='red', size=14, line=dict(color='black', width=1.5))
    ), secondary_y=False)

    # Volume bars on secondary y-axis (right)
    if 'volume' in datafix.columns:
        fig.add_trace(go.Bar(
            x=datafix.index, y=datafix['volume'],
            name='Volume',
            marker=dict(color='rgba(128,128,128,0.5)')
        ), secondary_y=True)

    fig.update_layout(
        title="Spread, Bollinger Bands, Buy/Sell Signals, and Volume",
        xaxis_title="Date",
        legend=dict(x=0.01, y=0.99),
        hovermode='x unified',
        template='plotly_white',
        width=1000,
        height=600
)

    fig.update_yaxes(title_text="Spread", secondary_y=False)
    fig.update_yaxes(title_text="Volume", secondary_y=True)

    fig.show()


    # --- Trade Performance Metrics ---
    trades = []
    in_trade = False
    trade_type = None
    entry_spread = None

    for idx, row in datafix.iterrows():
        sig = row['bb_signal']
        spread = row['spread']
        if not in_trade:
            if sig == 'buy':
                in_trade = True
                trade_type = 'long'
                entry_spread = spread
            elif sig == 'sell':
                in_trade = True
                trade_type = 'short'
                entry_spread = spread
        else:
            if trade_type == 'long' and sig == 'sell':
                ret = (spread - entry_spread) * 100 - 2 * fee
                trades.append({'type': 'long', 'ret': ret})
                in_trade = False
            elif trade_type == 'short' and sig == 'buy':
                ret = (entry_spread - spread) * 100 - 2 * fee
                trades.append({'type': 'short', 'ret': ret})
                in_trade = False

    trades_df = pd.DataFrame(trades)

    # Long trades
    long_trades = trades_df[trades_df['type'] == 'long']
    n_long = len(long_trades)
    n_long_win = (long_trades['ret'] > 0).sum()
    n_long_loss = (long_trades['ret'] <= 0).sum()
    avg_long_return = long_trades['ret'].mean() if n_long > 0 else np.nan
    pct_long_win = 100 * n_long_win / n_long if n_long > 0 else np.nan
    pct_long_loss = 100 * n_long_loss / n_long if n_long > 0 else np.nan

    # Short trades
    short_trades = trades_df[trades_df['type'] == 'short']
    n_short = len(short_trades)
    n_short_win = (short_trades['ret'] > 0).sum()
    n_short_loss = (short_trades['ret'] <= 0).sum()
    avg_short_return = short_trades['ret'].mean() if n_short > 0 else np.nan
    pct_short_win = 100 * n_short_win / n_short if n_short > 0 else np.nan
    pct_short_loss = 100 * n_short_loss / n_short if n_short > 0 else np.nan

    print(f"\nLong Trades: {n_long}, Avg Return: {avg_long_return:.2f}, % Correct: {pct_long_win:.2f}%, % Incorrect: {pct_long_loss:.2f}%")
    print(f"Short Trades: {n_short}, Avg Return: {avg_short_return:.2f}, % Correct: {pct_short_win:.2f}%, % Incorrect: {pct_short_loss:.2f}%")

    # Return just the key columns for inspection
    return datafix[['Close', 'spread', 'bb_upper', 'bb_lower', 'bb_middle', 'bb_signal', 'BB_Portfolio']]

# --- Run everything ---
datafix_summary = await main()
datafix = datafix_summary


Error 162, reqId 15: Historical Market Data Service error message:HMDS query returned no data: TSM   250718C00230000@SMART Trades, contract: Option(conId=742331880, symbol='TSM', lastTradeDateOrContractMonth='20250718', strike=230.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='TSM   250718C00230000', tradingClass='TSM')


No more bars available.


Error 162, reqId 15: Historical Market Data Service error message:HMDS query returned no data: TSM   250718C00240000@SMART Trades, contract: Option(conId=742331955, symbol='TSM', lastTradeDateOrContractMonth='20250718', strike=240.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='TSM   250718C00240000', tradingClass='TSM')


No more bars available.
Total Strategy $ Return (Bollinger Bands): $217.00



Long Trades: 24, Avg Return: 16.21, % Correct: 66.67%, % Incorrect: 33.33%
Short Trades: 25, Avg Return: 11.32, % Correct: 68.00%, % Incorrect: 32.00%


In [10]:


from statsmodels.tsa.stattools import adfuller

# Assuming your DataFrame is named datafix and has a 'spread' column
spread_series = datafix['spread'].dropna()

# Run the ADF test
adf_result = adfuller(spread_series)

print('ADF Statistic: %f' % adf_result[0])
print('p-value: %f' % adf_result[1])
for key, value in adf_result[4].items():
    print('Critical Value (%s): %.3f' % (key, value))

if adf_result[1] < 0.05:
    print("Result: The spread is likely stationary (reject H0).")
else:
    print("Result: The spread is likely non-stationary (fail to reject H0).")


ADF Statistic: -1.069088
p-value: 0.727240
Critical Value (1%): -3.432
Critical Value (5%): -2.862
Critical Value (10%): -2.567
Result: The spread is likely non-stationary (fail to reject H0).


In [11]:
import plotly.graph_objects as go

# --- Function to fetch latest N bars for each option ---
async def fetch_new_bars(symbol, lastTradeDateOrContractMonth, strike, right, n_bars=1):
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7496, clientId=np.random.randint(1000, 10000))
    option_contract = Option(
        symbol=symbol,
        lastTradeDateOrContractMonth=lastTradeDateOrContractMonth,
        strike=strike,
        right=right,
        exchange='SMART',
        currency='USD'
    )
    await ib.qualifyContractsAsync(option_contract)
    bars = await ib.reqHistoricalDataAsync(
        contract=option_contract,
        endDateTime='',
        durationStr=f'{n_bars} min',
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1
    )
    ib.disconnect()
    if not bars:
        print(f"No new bars returned for {symbol} {strike} {right}.")
        return pd.DataFrame()
    df = util.df(bars)
    df.set_index('date', inplace=True)
    df.rename(columns={'close': 'Close'}, inplace=True)  # Ensure 'Close' exists
    return df

# --- Function to update BB strategy with new data ---
def update_bb_strategy(datafix, datafix2, new_data1, new_data2, window=20, num_std=2):
    # Append new data if not already present
    datafix = pd.concat([datafix, new_data1[~new_data1.index.isin(datafix.index)]])
    datafix2 = pd.concat([datafix2, new_data2[~new_data2.index.isin(datafix2.index)]])
    datafix.index = datafix.index.tz_localize(None)
    datafix2.index = datafix2.index.tz_localize(None)
    # Update spread
    datafix["spread"] = datafix["Close"] - datafix2["Close"]
    # Update Bollinger Bands (only recalc for last window+1 rows for speed)
    recent = datafix.tail(window+1).copy()
    recent['bb_middle'] = recent['spread'].rolling(window=window).mean()
    recent['bb_std'] = recent['spread'].rolling(window=window).std()
    recent['bb_upper'] = recent['bb_middle'] + num_std * recent['bb_std']
    recent['bb_lower'] = recent['bb_middle'] - num_std * recent['bb_std']
    recent['prev_spread'] = recent['spread'].shift(1)
    recent['prev_bb_lower'] = recent['bb_lower'].shift(1)
    recent['prev_bb_upper'] = recent['bb_upper'].shift(1)
    buy_mask = (recent['prev_spread'] > recent['prev_bb_lower']) & (recent['spread'] < recent['bb_lower'])
    sell_mask = (recent['prev_spread'] < recent['prev_bb_upper']) & (recent['spread'] > recent['bb_upper'])
    recent['bb_signal'] = 'hold'
    recent.loc[buy_mask, 'bb_signal'] = 'buy'
    recent.loc[sell_mask, 'bb_signal'] = 'sell'
    datafix.update(recent)
    return datafix, datafix2

# --- Function to continue backtest from previous state ---
def continue_bb_backtest(datafix, initial_capital=1000, fee=1.50, multiplier=100):
    last_idx = datafix['BB_Portfolio'].last_valid_index()
    if last_idx is not None:
        capital = datafix.at[last_idx, 'BB_Portfolio']
        position = datafix.at[last_idx, 'BB_Position']
        if position == 1:
            entry_spread = datafix.loc[:last_idx][datafix['bb_signal'] == 'buy']['spread'].iloc[-1]
        else:
            entry_spread = None
        new_rows = datafix.loc[last_idx:].iloc[1:]
    else:
        capital = initial_capital
        position = 0
        entry_spread = None
        new_rows = datafix
    portfolio = []
    positions = []
    for idx, row in new_rows.iterrows():
        sig = row['bb_signal']
        spread = row['spread']
        if sig == 'buy' and position == 0:
            position = 1
            entry_spread = spread
            capital -= fee
        elif sig == 'sell' and position == 1:
            pnl = (spread - entry_spread) * multiplier
            capital += pnl
            capital -= fee
            position = 0
            entry_spread = None
        portfolio.append(capital)
        positions.append(position)
        datafix.at[idx, 'BB_Portfolio'] = capital
        datafix.at[idx, 'BB_Position'] = position
    return datafix

# --- Example usage: update with new data and plot ---
# Fetch new bars (example: 1 new bar)
new_data1 = await fetch_new_bars(symbol1, lastTradeDateOrContractMonth1, strike1, right1, n_bars=1)
new_data2 = await fetch_new_bars(symbol2, lastTradeDateOrContractMonth2, strike2, right2, n_bars=1)

# Update BB strategy DataFrame and backtest if we have new data
if not new_data1.empty and not new_data2.empty:
    datafix, datafix2 = update_bb_strategy(datafix, datafix2, new_data1, new_data2, window=20, num_std=2)
    datafix = continue_bb_backtest(datafix, initial_capital=1000, fee=1.50, multiplier=100)
    print("New bars found. Plotting latest data.")
else:
    print("No new bars found. Plotting last 30 bars of current datafix.")

# Always plot the last 30 bars (including new ones if present)
plot_df = datafix.tail(30)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df["spread"],
    mode='lines', name='Spread', line=dict(color='black', width=2)
))
fig.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['bb_upper'],
    mode='lines', name='BB Upper', line=dict(color='green', dash='dash')
))
fig.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['bb_lower'],
    mode='lines', name='BB Lower', line=dict(color='red', dash='dash')
))
fig.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['bb_middle'],
    mode='lines', name='BB Middle', line=dict(color='orange', dash='dot')
))
buy_signals = plot_df[plot_df["bb_signal"] == "buy"]
fig.add_trace(go.Scatter(
    x=buy_signals.index, y=buy_signals["spread"],
    mode='markers', name='Buy (on spread)',
    marker=dict(symbol='triangle-up', color='green', size=14, line=dict(color='black', width=1.5))
))
sell_signals = plot_df[plot_df["bb_signal"] == "sell"]
fig.add_trace(go.Scatter(
    x=sell_signals.index, y=sell_signals["spread"],
    mode='markers', name='Sell (on spread)',
    marker=dict(symbol='triangle-down', color='red', size=14, line=dict(color='black', width=1.5))
))
fig.update_layout(
    title="Spread, Bollinger Bands, and Buy/Sell Signals (Last 30 Bars)",
    xaxis_title="Date",
    yaxis_title="Spread",
    legend=dict(x=0.01, y=0.99),
    hovermode='x unified',
    template='plotly_white',
    width=1000,
    height=500
)
fig.show()


Error 321, reqId 4: Error validating request.-'bM' : cause - When specifying a unit, historical data request duration format is integer{SPACE}unit (S|D|W|M|Y)., contract: Option(conId=742331880, symbol='TSM', lastTradeDateOrContractMonth='20250718', strike=230.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='TSM   250718C00230000', tradingClass='TSM')


No new bars returned for TSM 230 C.


Error 321, reqId 4: Error validating request.-'bM' : cause - When specifying a unit, historical data request duration format is integer{SPACE}unit (S|D|W|M|Y)., contract: Option(conId=742331955, symbol='TSM', lastTradeDateOrContractMonth='20250718', strike=240.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='TSM   250718C00240000', tradingClass='TSM')


No new bars returned for TSM 240 C.
No new bars found. Plotting last 30 bars of current datafix.
